# Phylogenetic Inference with across site rate variation

This notebook gives an example on how to do phylogenetic inference using the PNUTS algorithm and across site rate variation.

First the MCPhylo package and the Random package are loaded. And the data is loaded.

In [1]:
using MCPhylo;
using Random;
Random.seed!(1234);

tree, data = make_tree_with_data("Example.nex");

data_dictionary = Dict{Symbol, Any}(
  :data => data
);

┌ Warning: Keyword datatype not understood, will be ignored
└ @ MCPhylo /home/gjaeger/.julia/packages/MCPhylo/KWPlY/src/Parser/ParseNexus.jl:92
┌ Warning: Keyword interleave not understood, will be ignored
└ @ MCPhylo /home/gjaeger/.julia/packages/MCPhylo/KWPlY/src/Parser/ParseNexus.jl:92


Create a model which uses discrete gamma rates for across site rate variaton.

In [2]:
model =  Model(
    data = Stochastic(3, (tree, eq_freq, rates) ->  PhyloDist(tree, eq_freq, [1.0], rates, Restriction), false),
    eq_freq = Stochastic(1, () -> Dirichlet(2,1)),
    tree = Stochastic(Node(), () -> TreeDistribution(CompoundDirichlet(1.0,1.0,0.100,1.0)), true),
    rates = Logical(1, (a)->discrete_gamma_rates(a,a,4)),
    a = Stochastic(()->Exponential(), true)
     )

Object of type "Model"
-------------------------------------------------------------------------------
a:
Object of type "Stochastic{Float64, var"#27#28", MCPhylo.NullUnivariateDistribution}"
NaN
-------------------------------------------------------------------------------
eq_freq:
Object of type "Stochastic{Vector{Float64}, var"#21#22", MCPhylo.NullUnivariateDistribution}"
Float64[]
-------------------------------------------------------------------------------
tree:
Object of type "Stochastic{GeneralNode{Float64, Int64}, var"#23#24", MCPhylo.NullUnivariateDistribution}"
Tree with root:
"no_name"
Length:
0.0
Height:
0.0
-------------------------------------------------------------------------------
rates:
Object of type "Logical{Vector{Float64}, var"#25#26"}"
Float64[]
-------------------------------------------------------------------------------
data:
Object of type "Stochastic{Array{Float64, 3}, var"#19#20", MCPhylo.NullUnivariateDistribution}"
Array{Float64, 3}(undef, 0, 0, 0)


Set the samplers for the tree, the equilibrium frequencies and the parameter for the discrete gamma rates.

In [3]:
scheme = [PNUTS(:tree, target=0.7, targetNNI=0.5),
          SliceSimplex(:eq_freq),
          Slice(:a, 1.0)
          ]

setsamplers!(model, scheme);

inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree => tree,
    :eq_freq=> rand(Dirichlet(2,1)),
    :data => data_dictionary[:data],
    :a => rand()
    ),
    ];


Run the MCMC. The statusbar is suppressed via the `verbose` argument, to avoid a cluttering of the output.

In [4]:
sim = mcmc(model, data_dictionary, inits, 5000, burnin=2500,thin=5, chains=1, trees=true, verbose=false)

Object of type "ModelChains"

Iterations = 2505:5000
Thinning interval = 5
Chains = 1
Samples per chain = 500
Burnin = 2500
[47.29790385561676 0.6054776401257185 … 2.143269560848256 -119.77231422891795; 41.910741847424305 0.6023453601328966 … 2.2974681799505676 -120.00079752732205; … ; 29.611462070141677 0.628138472303778 … 2.501859827301457 -120.37523039762013; 23.470047781218124 0.6671523105877593 … 2.1894329698700243 -119.22209094811562;;;]